In [ ]:
pip install xgboost

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance

import pickle
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier


Bad key text.latex.preview in file C:\anaconda3\envs\AIML\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\anaconda3\envs\AIML\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\anaconda3\envs\AIML\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You probably ne

In [2]:
def impFeature(indep_X,dep_Y,n):
        implist=[]
        featurexgb = XGBClassifier()
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        importFeature_lasso=Lasso(alpha=0.1)
     
        impmodellist=[featurexgb,RF,importFeature_lasso] 
       
        for i in   impmodellist:
            print(i)
            log_rfe = permutation_importance(estimator=i, n_features_to_select=n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
        return implist

In [3]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    

In [4]:
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 

In [5]:
def r2_prediction(regressor, X_test, y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)
    return r2

def Linear(X_train, y_train, X_test):
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def svm_linear(X_train, y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel='linear')
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def svm_NL(X_train, y_train, X_test):
    from sklearn.svm import SVR
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def Decision(X_train, y_train, X_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def random(X_train, y_train, X_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators=10, random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2


In [6]:
def imp_regression(r2_log, r2_svml, r2_svmnl, r2_knn, r2_nav, r2_des, r2_rf): 
    impdataframe = pd.DataFrame(index=['XGB', 'RF', 'Lasso'],
                                columns=['Linear', 'SVMl', 'SVMnl', 'KNN', 'Decision', 'Random'])

    for number, idex in enumerate(impdataframe.index):
        impdataframe['Linear'][idex] = r2_log[number]
        impdataframe['SVMl'][idex] = r2_svml[number]
        impdataframe['SVMnl'][idex] = r2_svmnl[number]
        impdataframe['KNN'][idex] = r2_knn[number]
        impdataframe['Decision'][idex] = r2_des[number]
        impdataframe['Random'][idex] = r2_rf[number]

    return impdataframe


In [7]:
dataset1 = pd.read_csv("Automobilecodedata.csv", index_col=None)
df2 = dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X = df2.drop('weight', axis=1)
dep_Y = df2['weight']

In [8]:
implist = impFeature(indep_X, dep_Y, 3)

r2_log = []
r2_svml = []
r2_svmnl = []
r2_knn = []
r2_nav = []
r2_des = []
r2_rf = []

for i in implist:
    X_train, X_test, y_train, y_test = split_scalar(i, dep_Y)
    
    r2 = Linear(X_train, y_train, X_test)
    r2_log.append(r2)
    
    r2 = svm_linear(X_train, y_train, X_test)
    r2_svml.append(r2)
    
    r2 = svm_NL(X_train, y_train, X_test)
    r2_svmnl.append(r2)
    
    r2 = knn(X_train, y_train, X_test)
    r2_knn.append(r2)
    
    r2 = Decision(X_train, y_train, X_test)
    r2_des.append(r2)
    
    r2 = random(X_train, y_train, X_test)
    r2_rf.append(r2)

# Now you can process your R² score lists, e.g.:
result = imp_regression(r2_log, r2_svml, r2_svmnl, r2_knn, r2_nav, r2_des, r2_rf)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


TypeError: permutation_importance() got an unexpected keyword argument 'n_features_to_select'